Import


In [67]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

bc = pd.read_csv('seatle.csv')

## Selection des batiments non residentiels

In [68]:
print(f"Nombre totale de batiments: {len(bc)}")
bc = bc[~bc['BuildingType'].str.contains('Multifamily', na=False)]
print(f"Nombre de batiments non residentiels: {len(bc)}")

Nombre totale de batiments: 3376
Nombre de batiments non residentiels: 1668


## Suppression des colonnes vides, avec des données manquantes, à zéro ou constantes
- Comments : Colonne vide
- Outlier : Valeurs manquantes
- YearsENERGYSTARCertified : Valeurs manquantes
- ThirdLargestPropertyUseType : Valeurs manquantes
- ThirdLargestPropertyUseTypeGFA : Valeurs manquantes
- SecondLargestPropertyUseType : Valeurs manquantes
- SecondLargestPropertyUseTypeGFA : Valeurs manquantes
- ENERGYSTARScore : Valeurs manquantes
- DataYear, City, State : Valeurs constantes
- OSEBuildingID, PropertyName, Address, TaxParcelIdentificationNumber, LargestPropertyUseType, LargestPropertyUseTypeGFA, DefaultData, CouncilDistrictCode, Latitude, Longitude : Valeurs non pertinentes,

In [69]:
print(f"Colonnes: {bc.shape[1]}")

bc.isnull().sum().sort_values(ascending=False)
# Suppression des colonnes avec plus de 500 données manquantes
bc = bc.loc[:, bc.isnull().sum() <= 500]
bc.isnull().sum().sort_values(ascending=False)

# Suppression des colonnes avec des données constantes
bc.columns[bc.nunique() == 1].tolist()
bc.drop((['DataYear','City','State']), axis=1, inplace=True)

# Suppression des colonnes non pertinentes
bc.drop((['OSEBuildingID','PropertyName','Address','TaxParcelIdentificationNumber','LargestPropertyUseType','LargestPropertyUseTypeGFA','DefaultData','CouncilDistrictCode','Latitude','Longitude']), axis=1, inplace=True)

# Suppression des colonnes avec des valeurs à 0
(bc == 0).sum().sort_values(ascending=False)
bc.drop((['SteamUse(kBtu)']), axis=1, inplace=True)

print(f"Colonnes: {bc.shape[1]}")

Colonnes: 46
Colonnes: 24


## Suppression des lignes avec des données manquantes

In [70]:
bc.isnull().sum().sort_values(ascending=False)
bc.dropna(subset=['SiteEUI(kBtu/sf)'], inplace=True)
bc.dropna(subset=['SiteEUIWN(kBtu/sf)'], inplace=True)
bc.isnull().sum().sort_values(ascending=False)

ZipCode                      16
BuildingType                  0
SourceEUI(kBtu/sf)            0
TotalGHGEmissions             0
ComplianceStatus              0
NaturalGas(kBtu)              0
NaturalGas(therms)            0
Electricity(kBtu)             0
Electricity(kWh)              0
SiteEnergyUseWN(kBtu)         0
SiteEnergyUse(kBtu)           0
SourceEUIWN(kBtu/sf)          0
SiteEUIWN(kBtu/sf)            0
PrimaryPropertyType           0
SiteEUI(kBtu/sf)              0
ListOfAllPropertyUseTypes     0
PropertyGFABuilding(s)        0
PropertyGFAParking            0
PropertyGFATotal              0
NumberofFloors                0
NumberofBuildings             0
YearBuilt                     0
Neighborhood                  0
GHGEmissionsIntensity         0
dtype: int64

## Traitement des quartiers et des codes postaux
- Certains quartiers sont en minuscules et 1 quartier "DELRIDGE NEIGHBORHOODS" peut etre modifié en "DELRIDGE"
- Sur les ZIPCODE manquants, on peut récupérer le quartier et ajouter le ZIPCODE le plus probable.

In [71]:
print(bc.groupby(['Neighborhood']).size().sort_values(ascending=False))

# Mettre les valeurs en MAJUSCULE
bc['Neighborhood'] = bc['Neighborhood'].str.upper()

# Modifier le quartier "DELRIDGE NEIGHBORHOODS")" en "DELRIDGE"
bc.loc[bc['Neighborhood'] == "DELRIDGE NEIGHBORHOODS", 'Neighborhood'] = "DELRIDGE"

print(bc.groupby(['Neighborhood']).size().sort_values(ascending=False))

Neighborhood
DOWNTOWN                  358
GREATER DUWAMISH          346
MAGNOLIA / QUEEN ANNE     151
LAKE UNION                148
NORTHEAST                 127
EAST                      121
NORTHWEST                  81
BALLARD                    64
NORTH                      58
CENTRAL                    51
SOUTHEAST                  46
DELRIDGE                   42
SOUTHWEST                  41
North                       9
Ballard                     6
Central                     5
Northwest                   5
Delridge                    4
DELRIDGE NEIGHBORHOODS      1
dtype: int64
Neighborhood
DOWNTOWN                 358
GREATER DUWAMISH         346
MAGNOLIA / QUEEN ANNE    151
LAKE UNION               148
NORTHEAST                127
EAST                     121
NORTHWEST                 86
BALLARD                   70
NORTH                     67
CENTRAL                   56
DELRIDGE                  47
SOUTHEAST                 46
SOUTHWEST                 41
dtype: int64


## Traitement des quartiers et des codes postaux
- Certains quartiers sont en minuscules et 1 quartier "DELRIDGE NEIGHBORHOODS" peut etre modifié en "DELRIDGE"
- Sur les ZIPCODE manquants, on peut récupérer le quartier et ajouter le ZIPCODE le plus probable.

In [ ]:
######## Traitement des ZipCode manquants ##########
prob = bc.groupby(['ZipCode', 'Neighborhood']).size().reset_index(name='Count')
prob = prob.loc[prob.groupby('Neighborhood')['Count'].idxmax()]
prob = prob[prob['Count'] > 25]
prob = prob.sort_values('Count', ascending=False)
print(prob)